In [5]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

In [ ]:
!pip install tslearn plotly

**Reading train.csv file which describes data**

In [6]:
df_train = pd.read_csv("/kaggle/input/asl-signs/train.csv")
print(df_train.shape)
df_train.head(10)

(94477, 4)


,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
5,train_landmark_files/26734/1000241583.parquet,26734,1000241583,duck
6,train_landmark_files/26734/1000255522.parquet,26734,1000255522,minemy
7,train_landmark_files/32319/1000278229.parquet,32319,1000278229,lips
8,train_landmark_files/37055/100035691.parquet,37055,100035691,flower
9,train_landmark_files/29302/100039661.parquet,29302,100039661,time


**Reading sign_to_prediction_index_map.json which contains integer label for each sign**

In [7]:
json_file_path = "/kaggle/input/asl-signs/sign_to_prediction_index_map.json"
with open(json_file_path, 'r') as j:
     sign_dict = json.loads(j.read())
        
ordered_signs = list(sign_dict.keys())
print(ordered_signs)

['TV', 'after', 'airplane', 'all', 'alligator', 'animal', 'another', 'any', 'apple', 'arm', 'aunt', 'awake', 'backyard', 'bad', 'balloon', 'bath', 'because', 'bed', 'bedroom', 'bee', 'before', 'beside', 'better', 'bird', 'black', 'blow', 'blue', 'boat', 'book', 'boy', 'brother', 'brown', 'bug', 'bye', 'callonphone', 'can', 'car', 'carrot', 'cat', 'cereal', 'chair', 'cheek', 'child', 'chin', 'chocolate', 'clean', 'close', 'closet', 'cloud', 'clown', 'cow', 'cowboy', 'cry', 'cut', 'cute', 'dad', 'dance', 'dirty', 'dog', 'doll', 'donkey', 'down', 'drawer', 'drink', 'drop', 'dry', 'dryer', 'duck', 'ear', 'elephant', 'empty', 'every', 'eye', 'face', 'fall', 'farm', 'fast', 'feet', 'find', 'fine', 'finger', 'finish', 'fireman', 'first', 'fish', 'flag', 'flower', 'food', 'for', 'frenchfries', 'frog', 'garbage', 'gift', 'giraffe', 'girl', 'give', 'glasswindow', 'go', 'goose', 'grandma', 'grandpa', 'grass', 'green', 'gum', 'hair', 'happy', 'hat', 'hate', 'have', 'haveto', 'head', 'hear', 'helic

**Function to remove null values**

In [12]:
def load_relevant_data(pq_path):
    data = pd.read_parquet(pq_path).fillna(0)
    return data

**Function to reshape data**

In [13]:
ROWS_PER_FRAME = 543  # number of landmarks per frame
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns).fillna(0)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [8]:
df_train['path'].values[3]

'train_landmark_files/25571/1000210073.parquet'

In [14]:
idx_plot = 3
path_show = "/kaggle/input/asl-signs/"+df_train['path'].values[idx_plot]
sign_plot = df_train['sign'].values[idx_plot]
path_example = path_show.replace("_", "_")

df = load_relevant_data(path_show)
df

,frame,row_id,type,landmark_index,x,y,z
0,17,17-face-0,face,0,0.495870,0.478694,-0.037412
1,17,17-face-1,face,1,0.492222,0.447209,-0.067939
2,17,17-face-2,face,2,0.492067,0.457237,-0.035722
3,17,17-face-3,face,3,0.480419,0.415996,-0.050779
4,17,17-face-4,face,4,0.492035,0.437453,-0.072314
...,...,...,...,...,...,...,...
6511,28,28-right_hand-16,right_hand,16,0.506396,0.868416,-0.139545
6512,28,28-right_hand-17,right_hand,17,0.323227,0.835990,-0.136632
6513,28,28-right_hand-18,right_hand,18,0.435733,0.848917,-0.156200
6514,28,28-right_hand-19,right_hand,19,0.476093,0.867098,-0.149442


In [37]:
#Let's explore one parquet file 
out=pd.read_parquet("/kaggle/input/asl-signs/"+df_train['path'].values[1],columns = ['x', 'y', 'z']).values.reshape(11,543,3)

In [38]:
#No. of frames in file
len(out)

11

In [39]:
#No. of points in one frame
len(out[0])

543

In [40]:
#No. of co-ordinates for one point
len(out[0][0])

3

In [41]:
path_show = "/kaggle/input/asl-signs/"+df_train['path'].values[1]
sign_show = df_train['sign'].values[1]

df_example = load_relevant_data_subset(path_show)

frames = df_example.shape[0]
keypoints = df_example.shape[1]
position = df_example.shape[2]

print("\nNumber of frames:", frames)
print("Keypoints:", keypoints)
print("X, Y Z postions:", position)
print("Total number of datapoints in this sequence:", np.prod(df_example.shape))


pose_landmarks = 33
face_landmarks = 468
right_hand_landmarks = 21
start_left_hand = face_landmarks
left_hand_landmarks = 21
start_right_hand = face_landmarks + left_hand_landmarks + pose_landmarks
total_landmarks = pose_landmarks + face_landmarks + right_hand_landmarks + left_hand_landmarks


print("\nPose landmarks:", pose_landmarks)
print("Face landmarks:", face_landmarks)
print("Right hand landmarks:", right_hand_landmarks)
print("Left hand landmarks:", left_hand_landmarks)
print("Total landmarks/keypoints: ", total_landmarks)


Number of frames: 11
Keypoints: 543
X, Y Z postions: 3
Total number of datapoints in this sequence: 17919

Pose landmarks: 33
Face landmarks: 468
Right hand landmarks: 21
Left hand landmarks: 21
Total landmarks/keypoints:  543


In [47]:
from tqdm import tqdm

max_sequence_length = 32
lip_marks = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324, 146, 91, 181, 84, 17, 314, 405, 321, 375]  

lips = lip_marks
left_hand = [*range(start_left_hand, start_left_hand+left_hand_landmarks, 1)]
right_hand = [*range(start_right_hand, start_right_hand+right_hand_landmarks, 1)]
meaningful_keypoints = lips + left_hand + right_hand
input_length = len(meaningful_keypoints)*3

def get_data(file_paths, y_sign):
    
    X = np.empty((file_paths.shape[0], max_sequence_length, len(meaningful_keypoints)*3), dtype=float)

    for i in tqdm(range(file_paths.shape[0])):
        file_name = "/kaggle/input/asl-signs/"+file_paths[i]
        data = load_relevant_data_subset(file_name)
        
        data = data[:, meaningful_keypoints]
        
        if data.shape[0] < max_sequence_length:
            rows = max_sequence_length - data.shape[0]
            data = np.append(np.zeros((rows, len(meaningful_keypoints), 3)), data, axis=0)
        elif data.shape[0] > max_sequence_length:
            data = data[-(max_sequence_length):]

        X[i] = data.reshape(max_sequence_length, len(meaningful_keypoints)*3, order='F')
        
        del data
        
    X = np.asarray(X).astype(np.float32)
        
    y = []
    for sign in y_sign:
        y.append(sign_dict[sign])

    y = np.array(y, dtype=int)

    return X, y

**LSTM Training**

In [42]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Input, optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import pad_sequences

In [44]:
def scaled_dot_product(q,k,v, softmax):
    #calculates Q . K(transpose)
    qkt = tf.matmul(q,k,transpose_b=True)
    #caculates scaling factor
    dk = tf.math.sqrt(tf.cast(q.shape[-1],dtype=tf.float32))
    scaled_qkt = qkt/dk
    softmax = softmax(scaled_qkt)
    
    z = tf.matmul(softmax,v)
    #shape: (m,Tx,depth), same shape as q,k,v
    return z

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model,num_of_heads):
        super(MultiHeadAttention,self).__init__()
        self.d_model = d_model
        self.num_of_heads = num_of_heads
        self.depth = d_model//num_of_heads
        self.wq = [tf.keras.layers.Dense(self.depth) for i in range(num_of_heads)]
        self.wk = [tf.keras.layers.Dense(self.depth) for i in range(num_of_heads)]
        self.wv = [tf.keras.layers.Dense(self.depth) for i in range(num_of_heads)]
        self.wo = tf.keras.layers.Dense(d_model)
        self.softmax = tf.keras.layers.Softmax()
        
    def call(self,x):
        
        multi_attn = []
        for i in range(self.num_of_heads):
            Q = self.wq[i](x)
            K = self.wk[i](x)
            V = self.wv[i](x)
            multi_attn.append(scaled_dot_product(Q,K,V, self.softmax))
            
        multi_head = tf.concat(multi_attn,axis=-1)
        multi_head_attention = self.wo(multi_head)
        return multi_head_attention

In [48]:
# a single dense block followed by a normalization block and relu activation
def dense_block(units):
    fc = layers.Dense(units)
    norm = layers.LayerNormalization()
    act = layers.Activation("relu")
    drop = layers.Dropout(0.05)
    return lambda x: drop(act(norm(fc(x))))

# transformer blocks
def transformer_block(key_dim, x):
    mha = MultiHeadAttention(key_dim, 8)(x)
    add1 = layers.add([mha, x])
    norm1 = layers.LayerNormalization()(add1)

    fc = layers.Dense(key_dim, activation="relu")(norm1)
    add2 = tf.math.add(fc, norm1)
    norm2 = layers.LayerNormalization()(add2)

    return norm2

# the final dense block for the classification
def classifier_lstm(units):
    lstm = layers.LSTM(units)
    out = layers.Dense(250, activation="softmax", name="outputs")
    return lambda x: out(lstm(x))
    
def classifier_transformer():
    dense = layers.Dense(256, activation="relu")
    drop = layers.Dropout(0.1)
    
    out = layers.Dense(250, activation="softmax", name="outputs")
    return lambda x: out(drop(dense(x)))

inputs = tf.keras.Input(shape=(None, input_length), ragged=True)
# choose the number of nodes per layer
embedding_units = [256, 128, 256] # tune this
transformer_units = []#, 512, 512]

# # dense encoder model
x = inputs
for n in embedding_units:
    x = dense_block(n)(x)
    
for t in transformer_units:
    x = transformer_block(t, x)

# classifier layer
if len(transformer_units) > 0:
    # Pooling
    x = tf.math.reduce_sum(x, axis=1)
    out = classifier_transformer()(x)
else:
    out = classifier_lstm(embedding_units[-1])(x)


model = tf.keras.Model(inputs=inputs, outputs=out)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 246)]       0         
                                                                 
 dense (Dense)               (None, None, 256)         63232     
                                                                 
 layer_normalization (LayerN  (None, None, 256)        512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, None, 256)         0         
                                                                 
 dropout (Dropout)           (None, None, 256)         0         
                                                                 
 dense_1 (Dense)             (None, None, 128)         32896     
                                                             

In [ ]:
# add a decreasing learning rate scheduler to help convergence
batch_size = 256
validation_percentage = 0.05
steps_per_epoch = int(94477*(1-validation_percentage)) // batch_size
boundaries = [steps_per_epoch * n for n in [23, 35, 45, 53, 60]]
# print(boundaries)
values = [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8]
lr_sched = optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

optimizer = optimizers.Adam(lr_sched)
# optimizer = optimizers.Adam()

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(name="loss"),
              metrics=["accuracy","sparse_top_k_categorical_accuracy"])

In [ ]:
def get_callbacks():
    return [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience = 10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = "val_accuracy",
            factor = 0.2,
            patience = 5
        ),
    ]

In [ ]:
mode='training'

In [ ]:
if mode == "training":
    file_paths = df_train['path'].values#[:1000]
    y_sign = df_train['sign'].values#[:1000]
    X, y = get_data(file_paths, y_sign)

    X, X_val, y, y_val = train_test_split(X, y, test_size=validation_percentage, random_state=123)

#     print(X.shape)
#     print(y.shape)

    history = model.fit(X, y, 
                    epochs=5,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val),
                    verbose=2,
                    callbacks=[get_callbacks()]
                   )

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

In [ ]:
if mode == "training":
    import matplotlib.pyplot as plt
    fig = plt.figure()

    plt.plot(val_accuracy, label='val_accuracy')

    plt.plot(accuracy, label='accuracy')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    # plt.ylim([0.5, 1])
    plt.legend()

In [ ]:
lips = lip_marks
left_hand = [*range(start_left_hand, start_left_hand+left_hand_landmarks, 1)]
right_hand = [*range(start_right_hand, start_right_hand+right_hand_landmarks, 1)]
meaningful_keypoints = lips + left_hand + right_hand


def get_inference_model(model):
    inputs = tf.keras.Input(shape=(ROWS_PER_FRAME,3), name="inputs")
    
    # drop most of the face mesh
    x = tf.gather(inputs, meaningful_keypoints, axis=1)

    # fill nan
    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)

    # flatten landmark xyz coordinates ()
    x = tf.concat([x[...,i] for i in range(3)], -1)

    x = tf.expand_dims(x,0)
    
    # call trained model
    out = model(x)
    
    # explicitly name the final (identity) layer for the submission format
    outputs = layers.Activation("linear", name="outputs")(out)
    
    inference_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    inference_model.compile(loss="sparse_categorical_crossentropy",
                            metrics="accuracy")
    return inference_model

In [ ]:
inference_model = get_inference_model(model)
inference_model.summary(expand_nested=True)

In [ ]:
if mode == "training":
    converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
    tflite_model = converter.convert()

In [ ]:
if mode == "training":
    with open('/kaggle/working/model.tflite', 'wb') as f:
        f.write(tflite_model)

In [ ]:
if mode == "submission":
    !cp -r /kaggle/input/tflite-model/model.tflite ./

In [ ]:
!zip submission.zip model.tflite